# 날씨누리에서 강수일시 파악하기

# 환경설정: 터미널에서 (rain) 나오고 있는지 확인

## 1시간 강수량이 0이나 . 인 경우를 제외한 데이터만 출력(기온, 습도, 해면기압 등은 데이터 맞는지 확인용임)

### 수정대상 
### 1. with open()여기 들어갈 csv파일명
### 2. day in range()에 분석 대상 월 말일+1 로 수정
### 3. tm = f"2025.0%."이부분 해당월로 수정

In [3]:
import requests
import csv
from bs4 import BeautifulSoup

url = "https://www.weather.go.kr/w/observation/land/aws-obs.do"

# 콘솔에 헤더 출력
print("조회시간\t시:분\t1시간 강수량(mm)\t기온(℃)\t습도(%)\t해면기압(hPa)")

# CSV 파일 열기 (UTF-8 인코딩 및 새 줄 문자 지정)
with open("rain2503.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    # CSV 파일에 헤더 쓰기
    writer.writerow(["조회시간", "시:분", "1시간 강수량(mm)", "기온(℃)", "습도(%)", "해면기압(hPa)"])
    
    for day in range(1, 32):
        for hour in range(1, 25):
            tm = f"2025.03.{day:02d} {hour:02d}:00"
            params = {
                "db": "MINDB_01M",
                "tm": tm,
                "stnId": "565",
                "sidoCode": "4100000000",
                "sort": "",
                "config": ""
            }
            
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                table_rows = soup.select("table tbody tr")
                
                if table_rows:
                    for row in reversed(table_rows):
                        cols = row.find_all("td")
                        if len(cols) > 10:
                            time_val = cols[0].text.strip()       # 시:분
                            rainfall = cols[2].text.strip()         # 1시간 강수량(mm)
                            
                            # 1시간 강수량이 0 혹은 "."인 경우는 데이터 처리에서 제외
                            if rainfall not in["0", "."]:
                                temperature = cols[4].text.strip()  # 기온(℃)
                                humidity = cols[10].text.strip()    # 습도(%)
                                pressure = cols[11].text.strip()    # 해면기압(hPa)
                                
                                # 출력할 데이터 형식 지정
                                output = f"{tm}\t{time_val}\t{rainfall}\t{temperature}\t{humidity}\t{pressure}"
                                print(output)
                                writer.writerow([tm, time_val, rainfall, temperature, humidity, pressure])
                else:
                    message = f"{tm}\t데이터 없음"
                    print(message)
                    writer.writerow([tm, "데이터 없음"])
            else:
                error = f"{tm}\t페이지 요청 실패: {response.status_code}"
                print(error)
                writer.writerow([tm, f"페이지 요청 실패: {response.status_code}"])


조회시간	시:분	1시간 강수량(mm)	기온(℃)	습도(%)	해면기압(hPa)
2025.03.02 16:00	15:35	0.5	11.9	51	1,014
2025.03.02 16:00	15:36	0.5	11.8	51	1,014
2025.03.02 16:00	15:37	0.5	11.8	51	1,014
2025.03.02 16:00	15:38	0.5	11.6	53	1,014.1
2025.03.02 16:00	15:39	0.5	11.6	53	1,014
2025.03.02 16:00	15:40	0.5	11.6	52	1,014
2025.03.02 16:00	15:41	0.5	11.6	53	1,014.1
2025.03.02 16:00	15:42	0.5	11.5	54	1,014.1
2025.03.02 16:00	15:43	0.5	11.3	55	1,014.1
2025.03.02 16:00	15:44	0.5	11.4	54	1,014.1
2025.03.02 16:00	15:45	0.5	11.3	54	1,014.2
2025.03.02 16:00	15:46	0.5	11.2	56	1,014.2
2025.03.02 16:00	15:47	0.5	11.1	56	1,014.1
2025.03.02 16:00	15:48	0.5	11.1	56	1,014.1
2025.03.02 16:00	15:49	0.5	11.2	55	1,014.2
2025.03.02 16:00	15:50	0.5	11.1	56	1,014.2
2025.03.02 16:00	15:51	0.5	11	57	1,014.2
2025.03.02 16:00	15:52	0.5	10.8	59	1,014.2
2025.03.02 16:00	15:53	0.5	10.7	60	1,014.2
2025.03.02 16:00	15:54	0.5	10.6	61	1,014.2
2025.03.02 16:00	15:55	0.5	10.5	62	1,014.2
2025.03.02 16:00	15:56	0.5	10.4	62	1,014.3
2025.03.02 16:00	15:57	

## 위의 출력 결과에서 분석에 적합하도록 일시분을 칼럼에 추가하여 다시 저장 
### 수정대상 
### 1. data = pd.read_csv('rain2504.csv') csv파일명
### 2. data.to_csv('rain2504_updated.csv', index=False, encoding='utf-8')csv파일명

In [4]:
import pandas as pd

# Load the existing CSV file
data = pd.read_csv('rain2503.csv')

# Extract the 9th and 10th digits from the first column (조회시간) and the 4-digit time from the second column (시:분)
data['일시분'] = data['조회시간'].str[8:10] + data['시:분'].str.replace(':', '')

# Save the updated DataFrame back to a CSV file
data.to_csv('rain2503_updated.csv', index=False, encoding='utf-8')

# Preview the updated data
print(data.head())


               조회시간    시:분  1시간 강수량(mm)  기온(℃)  습도(%) 해면기압(hPa)     일시분
0  2025.03.02 16:00  15:35          0.5   11.9     51     1,014  021535
1  2025.03.02 16:00  15:36          0.5   11.8     51     1,014  021536
2  2025.03.02 16:00  15:37          0.5   11.8     51     1,014  021537
3  2025.03.02 16:00  15:38          0.5   11.6     53   1,014.1  021538
4  2025.03.02 16:00  15:39          0.5   11.6     53     1,014  021539


##  이 코드의 목적: "CSV의 시간 데이터와 실제 파일명에 있는 시간 정보가 일치하는지 검증"
## CSV 파일 로드 → 2. 폴더 31개에서 파일명 추출 → 3. 두 데이터 비교 → 4. 최종 CSV 저장
### 수정대상 
### 1. data = pd.read_csv('rain2503_updated.csv')
### 2. base_directory = "D:/25.03_daily_prep"
### 3. data.to_csv('rain2412_final.csv',)


In [6]:
import pandas as pd
import os

# Load the existing CSV file
data = pd.read_csv('rain2503_updated.csv')

# Extract '일시분' column from the CSV
data['일시분'] = data['조회시간'].str[8:10] + data['시:분'].str.replace(':', '')

# Base directory and folder paths
base_directory = "D:/25.03_daily_prep"
folder_paths = [f"{base_directory}/{str(i).zfill(2)}" for i in range(1, 32)]

# List to store extracted 6-digit numbers for comparison
six_digit_numbers = []

# Iterate through each folder and extract 6-digit numbers from file names
for folder_path in folder_paths:
    try:
        file_names = os.listdir(folder_path)
        for file_name in file_names:
            if len(file_name) >= 13:  # Ensure file name is long enough
                six_digit_number = file_name[6:8] + file_name[9:13]  # Extract and combine parts
                six_digit_numbers.append(six_digit_number)
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")

# Add a new column '일시분vs6digit' to compare '일시분' with extracted numbers
data['일시분vs6digit'] = data['일시분'].apply(
    lambda x: x if x in six_digit_numbers else 0
)

# Save the updated DataFrame back to a CSV file
data.to_csv('rain2503_final.csv', index=False, encoding='utf-8')

# Preview the updated data
print(data.head())


               조회시간    시:분  1시간 강수량(mm)  기온(℃)  습도(%) 해면기압(hPa)     일시분  \
0  2025.03.02 16:00  15:35          0.5   11.9     51     1,014  021535   
1  2025.03.02 16:00  15:36          0.5   11.8     51     1,014  021536   
2  2025.03.02 16:00  15:37          0.5   11.8     51     1,014  021537   
3  2025.03.02 16:00  15:38          0.5   11.6     53   1,014.1  021538   
4  2025.03.02 16:00  15:39          0.5   11.6     53     1,014  021539   

  일시분vs6digit  
0           0  
1           0  
2           0  
3           0  
4           0  


## csv 파일 열어서 일시분vs6digit 칼럼이 0인 경우를 필터 해제 
### 0이 아님 → "해당 시간대의 파일이 실제로 존재함"
### 0임 → "해당 시간대의 파일이 없음"
## 0이 아닌 파일은 강수량이 0초과 하면서도 라이다 측정이 된 경우이므로 삭제한다